# PFS - Reading

Support for PFS is limited to files with unique subsections. This applies to all files in the MIKE FM family.

It is only possible to read files, there is no support for modifying or writing files at this stage.

In [1]:
from mikeio import Pfs

pfs = Pfs("../tests/testdata/lake.sw")

It is possible to navigate to each section and keyword in the pfs file:

In [2]:
pfs.data.DOMAIN.file_name

'.\\Lake_Mesh.mesh'

In [3]:
pfs.data.MODULE_SELECTION.mode_of_spectral_wave_module

2

In [4]:
pfs.SW.SPECTRAL.number_of_directions

16

In [5]:
pfs.SW.SPECTRAL.maximum_threshold_frequency

0.5959088268863615

Some common information is also available in tabular form (dataframe).

In [6]:
df = pfs.SW.OUTPUTS.to_dataframe()
df

,file_name,include,type,format,first_time_step,last_time_step,use_end_time,time_step_frequency
1,Wave_parameters.dfsu,1,1,2,0,450,1,10
2,Wave_line.dfs1,0,1,1,0,450,1,10
3,Waves_x20km_y20km.dfs0,1,1,0,0,450,1,1
4,spectrum_x20km_y20km.dfsu,1,4,0,0,450,1,10
